In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import albumentations as albu
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import KFold
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import timm
from torchmetrics import Metric
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import random

In [2]:
class NMAE(Metric):
    def __init__(self):
        super().__init__()
        self.add_state("mae", default=[], dist_reduce_fx=None)
        self.add_state("deno", default=[], dist_reduce_fx=None)
        
    def update(self, preds: torch.Tensor, target: torch.Tensor):
        assert preds.shape == target.shape
        mae = torch.abs(target-preds)
        deno = torch.abs(target)
        self.mae.append(mae)
        self.deno.append(deno)
    def compute(self):
        return torch.mean(torch.cat(self.mae)) / torch.mean(torch.cat(self.deno))

In [57]:
# model_type = "efficientnet_B0"
model_type = "efficientnetv2_rw_m"

In [58]:
#processing outlier

if model_type ==  "efficientnet_B0":
    preprocessing_train_imgs = glob("D:/DACON_GROWTH/PREPROCESSING-TRAIN_224/*")
    preprocessing_test_imgs = glob("D:/DACON_GROWTH/PREPROCESSING-TEST_224/*")
    augmetation_imgs = glob("D:/DACON_GROWTH/AUGMENTATION/SIZE224/*")

if model_type ==  "efficientnetv2_rw_m":
    preprocessing_train_imgs = glob("D:/DACON_GROWTH/PREPROCESSING-TRAIN_416/*")
    preprocessing_test_imgs = glob("D:/DACON_GROWTH/PREPROCESSING-TEST_416/*")
    augmetation_imgs = glob("D:/DACON_GROWTH/AUGMENTATION/SIZE416/*")
    
outlier_imgs = []
outlier_path = []
train_path = []
train_imgs = []
for path in tqdm(preprocessing_train_imgs):
    img_file = os.path.basename(path)
    img_name = img_file.split(".")[0]
    case = img_name.split("_")[0]
    if case in ["CASE59", "CASE11", "CASE12", "CASE13", "CASE14"]:
        continue
    elif img_name in ["CASE05_21", "CASE10_41", "CASE45_16", "CASE45_17",
                     "CASE73_04", "CASE73_10", "CASE73_11", "CASE73_14"]:        
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        outlier_path.append(path)
        outlier_imgs.append(img)
    elif case in ["CASE16", "CASE22", "CASE23"]:
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        outlier_path.append(path)
        outlier_imgs.append(img)
    else:
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        train_path.append(path)
        train_imgs.append(img)
        
test_path = []
test_imgs = []
for path in tqdm(preprocessing_test_imgs):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    test_path.append(path)
    test_imgs.append(img)

augment_path = []
augment_imgs = []
for path in tqdm(augmetation_imgs):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    augment_path.append(path)
    augment_imgs.append(img)
    
augment_path = augment_path + outlier_path
augment_imgs = augment_imgs + outlier_imgs

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 492/492 [00:02<00:00, 179.93it/s]


In [5]:
main_path = "D:/DACON_GROWTH"
train_data = glob(main_path+"/train/*/meta/*.csv")
train_label = glob(main_path+"/train/*/*.csv")
test_data = glob(main_path+"/test/meta/*.csv")

train_df = []
for i in tqdm(train_data):
    name = i.split("\\")[-1].split(".")[0]
    df = pd.read_csv(i)
    df["이미지"] = name
    case = name.split("_")[0]
    label = pd.read_csv(f"D:/DACON_GROWTH/train/{case}/label.csv")
    label_name = [i.split(".")[0] for i in label.img_name]
    label.img_name = label_name
    leaf_weight = label[label.img_name == name].leaf_weight.values[0]
    df["무게"] = leaf_weight
    df = df.replace([np.inf, -np.inf], np.nan)
    train_df.append(df)

train_df = pd.concat(train_df, ignore_index=True)

img2weight = {}
for i in tqdm(train_df["이미지"].unique()):
    temp = train_df[train_df["이미지"]==i]
    weight = temp["무게"].values[0]
    img2weight[i] = weight
    
train_df["시간2"] = train_df["시간"].apply(lambda x: x.split(" ")[-1])

golden_dict ={}
for e, i in enumerate(train_df["시간2"].unique()):
       golden_dict[i] = e
train_df["순서"] = train_df["시간2"].apply(lambda x : golden_dict[x])

train_df2 = train_df[(train_df["순서"] > 880) & (train_df["순서"] < 930)]

train_dfx = train_df2.copy()
train_dfx.drop(columns=["시간", "시간2"], inplace=True)
train_dfx.fillna(0., inplace=True)

img2data = {}
for i in tqdm(train_dfx['이미지'].unique()):
    temp = train_dfx[train_dfx["이미지"]==i]
    temp.drop(columns = ["무게", "이미지"], inplace=True)
    img2data[i] = np.array(temp.values, dtype=np.float32)
    
test_df = []
for i in tqdm(test_data):
    name = i.split("\\")[-1].split(".")[0]
    df = pd.read_csv(i)
    df["이미지"] = name
    df = df.replace([np.inf, -np.inf], np.nan)
    test_df.append(df)
    
test_df = pd.concat(test_df, ignore_index=True)
test_df["시간2"] = test_df["시간"].apply(lambda x: x.split(" ")[-1])
test_df["순서"] = test_df["시간2"].apply(lambda x : golden_dict[x])
test_df2 = test_df[(test_df["순서"] > 880) & (test_df["순서"] < 930)]

test_dfx = test_df2.copy()
test_dfx.drop(columns=["시간", "시간2"], inplace=True)
test_dfx.fillna(0., inplace=True)

img2test = {}
for i in tqdm(test_dfx['이미지'].unique()):
    temp = test_dfx[test_dfx["이미지"]==i]
    temp.drop(columns = "이미지", inplace=True)
    img2test[i] = np.array(temp.values, dtype=np.float32)

  0%|                                                                                                                                                                                                       | 0/1592 [00:00<?, ?it/s]D:\meta2\venv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 460/460 [00:01<00:00, 338.06it/s]


In [28]:
class PYTORCH_DATASET(nn.Module):
    def __init__(self, path: list, img2weight: dict, imgs: np.array, img2data: dict,  \
                 transform=None):
        super(PYTORCH_DATASET, self).__init__()
        self.path = path
        self.imgs = imgs

        self.transform = transform
        self.img2weight = img2weight
        self.img2data = img2data

    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        before_img = self.imgs[idx]
        img_path = self.path[idx]
        img_path = img_path.split("\\")[-1].split(".")[0]
        df_index = self.img2data[img_path]
        df_label = self.img2weight[img_path]

        after_img = self.transform(image=before_img)["image"]

        return {"사진": after_img, "정보": df_index, "상표": df_label}


class PYTORCH_PREDICTSET(nn.Module):
    def __init__(self, path: list,  imgs: np.array, img2data: dict, \
                 transform=None):
        super(PYTORCH_PREDICTSET, self).__init__()
        self.path = path
        self.imgs = imgs

        self.transform = transform
        self.img2data = img2data

    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        before_img = self.imgs[idx]
        img_path = self.path[idx]
        img_path = img_path.split("\\")[-1].split(".")[0]
        df_index = self.img2data[img_path]
        after_img = self.transform(image=before_img)["image"]
        return {"사진": after_img, "정보": df_index}

class PYTORCH_AUGMENTSET(nn.Module):
    def __init__(self, aug_imgs: np.array, transform_ori = None, transform_aug=None):
        super(PYTORCH_AUGMENTSET, self).__init__()
        self.aug_imgs = aug_imgs
        self.transform_ori = transform_ori
        self.transform_aug = transform_aug

    def __len__(self):
        return len(self.aug_imgs)

    def __getitem__(self, idx):
        before_img = self.aug_imgs[idx]
        after_img = self.transform_aug(image=before_img)["image"]
        before_img = self.transform_ori(image=before_img)["image"]
        temp_data = torch.FloatTensor(torch.zeros([49,19]))

        return {"원본": before_img, "확장": after_img, "정보": temp_data}

In [59]:
if model_type ==  "efficientnet_B0":
    train_image_size = 224
    test_image_size = 224
if model_type == "efficientnetv2_rw_m":
    train_image_size = 320
    test_image_size = 416
    
train_mode= albu.Compose([
    albu.Resize(train_image_size, train_image_size),
    albu.RandomResizedCrop(height=train_image_size, width=train_image_size,
                           scale=(0.25, 1.0), ratio=(0.75, 1.3333333333333333),
                           interpolation=1, p=0.9),
    albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1,
                          rotate_limit=30, interpolation=1, border_mode=0, value=0, p=0.25),
    albu.HorizontalFlip(p=0.5),
    albu.VerticalFlip(p=0.5),
    albu.OneOf([
        albu.MotionBlur(p=.2),
        albu.MedianBlur(blur_limit=3, p=0.1),
        albu.Blur(blur_limit=3, p=0.1),
    ], p=0.25),
    albu.OneOf([
        albu.CLAHE(clip_limit=2),
        albu.Sharpen(),
        albu.Emboss(),
        albu.RandomBrightnessContrast(),
    ], p=0.25),
    albu.CoarseDropout(max_holes=8, max_height=8, max_width=8, fill_value=0, p=0.25),
    albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
    ToTensorV2(),
])

test_mode = albu.Compose([
    albu.Resize(test_image_size, test_image_size),
    albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
    ToTensorV2(),
])

import torch
import numpy as np
from torch import nn
from einops.layers.torch import Rearrange


class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class MixerBlock(nn.Module):

    def __init__(self, dim, num_sequence, token_dim, channel_dim, dropout=0.):
        super().__init__()

        self.token_mix = nn.Sequential(
            nn.LayerNorm(dim),
            Rearrange('b n d -> b d n'),
            FeedForward(num_sequence, token_dim, dropout),
            Rearrange('b d n -> b n d')
        )

        self.channel_mix = nn.Sequential(
            nn.LayerNorm(dim),
            FeedForward(dim, channel_dim, dropout),
        )

    def forward(self, x):
        x = x + self.token_mix(x)

        x = x + self.channel_mix(x)

        return x

class MLPMixer(nn.Module):

    def __init__(self):
        super().__init__()

        self.dim = 19
        self.num_sequence = 49
        self.token_dim = 64
        self.channel_dim = 128
        self.mixer_blocks = nn.ModuleList([])
        self.depth = 1
        for _ in range(self.depth):
            self.mixer_blocks.append(MixerBlock(self.dim, self.num_sequence, self.token_dim, self.channel_dim))
        self.layer_norm = nn.LayerNorm(self.dim)

    def forward(self, x):
        for mixer_block in self.mixer_blocks:
            x = mixer_block(x)

        x = self.layer_norm(x)

        x = x.mean(dim=1)

        return x
    
class EFFB0_MIXER(nn.Module):
    def __init__(self):
        super(EFFB0_MIXER, self).__init__()
        self.model = timm.create_model("efficientnet_b0", pretrained=True, \
                                       num_classes=1)
        self.meta = MLPMixer()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(1299, 256)
        self.output = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.9)
        self.dropout1 = nn.Dropout(0.65)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0)

    def forward(self,  x, x2, training):
        x = self.model.forward_features(x)
        x = self.avg(x)
        x = x.flatten()
        x = x.view(-1,  1280)
        x = self.dropout(x)
        x2 = self.meta(x2)
        if training:
            c = random.choice([0, 0, 0, 1, 2, 3])
            if c == 0:
                pass
            elif c == 1:
                x2 = self.dropout1(x2)
            elif c == 2:
                x2 = self.dropout2(x2)
            elif c == 3:
                x2 = self.dropout3(x2)
        x = torch.cat([x, x2], dim=-1)
        x = self.linear(x)
        x = self.relu(x)
        x = self.output(x)
        return x
    
class EFFV2M_MIXER(nn.Module):
    def __init__(self):
        super(EFFV2M_MIXER, self).__init__()
        self.model = timm.create_model("efficientnetv2_rw_m", pretrained=True, \
                                       num_classes=1)
        self.meta = MLPMixer()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(2171, 128)
        self.output = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.9)
        self.dropout1 = nn.Dropout(0.65)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0)

    def forward(self,  x, x2, training):
        x = self.model.forward_features(x)
        x = self.avg(x)
        x = x.flatten()
        x = x.view(-1,  2152)
        x = self.dropout(x)
        x2 = self.meta(x2)
        if training:
            c = random.choice([0, 0, 0, 1, 2, 3])
            if c == 0:
                pass
            elif c == 1:
                x2 = self.dropout1(x2)
            elif c == 2:
                x2 = self.dropout2(x2)
            elif c == 3:
                x2 = self.dropout3(x2)
        x = torch.cat([x, x2], dim=-1)
        x = self.linear(x)
        x = self.relu(x)
        x = self.output(x)
        return x
    


In [38]:
from pytorch_lightning.trainer.supporters import CombinedLoader

class LIGHTNING_MODEL(pl.LightningModule):
    def __init__(self, model, train_datasest, augment_dataset, predict_dataset, \
                train_batch, augment_batch, predict_batch):
        super(LIGHTNING_MODEL, self).__init__()
        self.model = model()
        self.train_nmae = NMAE()
        self.valid_nmae = NMAE()
        self.augment_dataset= augment_dataset
        self.train_dataset = train_dataset
        self.predict_dataset = predict_dataset
        self.train_batch = train_batch
        self.augment_batch = augment_batch
        self.predict_batch = predict_batch
        
    def forward(self, x):
        output = self.model(x)
        return output
    
    def train_dataloader(self):
        loaders = {"a": DataLoader(self.train_dataset, shuffle=True, batch_size=self.train_batch, num_workers=0), \
                   "b": DataLoader(self.augment_dataset, shuffle=True, batch_size=self.augment_batch, num_workers=0)}
        combined_loaders = CombinedLoader(loaders, mode="max_size_cycle")
        return combined_loaders

    def predict_dataloader(self):
        return DataLoader(self.predict_dataset, shuffle=False, batch_size=self.predict_batch, num_workers=0)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        ori_batch = train_batch["a"]
        aug_batch = train_batch["b"]
        img = ori_batch["사진"]
        data = ori_batch["정보"]
        label = ori_batch["상표"]
        
        ori = aug_batch["원본"]
        ext = aug_batch["확장"]
        temp_data = aug_batch["정보"]

        pred = self.model(img, data, training=True)
        pred = pred.view(-1)
        label = label.view(-1)
        
        train_loss = nn.L1Loss()(pred, label)
        self.log("train_loss", train_loss)
        
        train_nmae = self.train_nmae(pred, label)
        self.log('train_NMAE', train_nmae)
        
        with torch.no_grad():
            ori_out = self.model(ori, temp_data, training=False)
            ext_out = self.model(ext, temp_data, training=False)
        con_loss = nn.L1Loss()(ori_out, ext_out)
        con_loss = 0.25 * con_loss 
        self.log('con_loss', con_loss)
        loss = train_loss + con_loss
        self.log('loss', loss)
        return train_loss

    def training_epoch_end(self, outputs):
        train_nmae = self.train_nmae.compute()
        self.log("epoch_train_nmae", train_nmae)
        self.train_nmae.reset()
        print(f"training nmae: {train_nmae:.4}, epoch: {self.trainer.current_epoch}")


    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        img = batch["사진"]
        data = batch["정보"]
        pred = self.model(img, data, training=False).unsqueeze(0)
        return pred

In [60]:
train_dataset = PYTORCH_DATASET(path=train_path, img2weight=img2weight, \
                                imgs=train_imgs, img2data=img2data, \
                                transform=train_mode)

augment_dataset = PYTORCH_AUGMENTSET(aug_imgs=augment_imgs, transform_ori=test_mode, transform_aug=train_mode)

test_dataset = PYTORCH_PREDICTSET(path=test_path, imgs=test_imgs, img2data=img2test, \
                                    transform=test_mode)

In [13]:
#check
print(len(train_path))
print(len(augment_imgs))
print(len(test_path))
print(len(train_path)//len(augment_imgs)+1)

1362
524
460
3


In [45]:
if model_type == "efficientnet_B0":
    model = LIGHTNING_MODEL(EFFB0_MIXER, train_dataset, augment_dataset, test_dataset, \
                           32, 10, 32)
    check_path =  f"D:/DACON_GROWTH/EFFB0_MiXER"
    if not os.path.exists(check_path):
        os.makedirs(check_path)
        
if model_type == "efficientnetv2_rw_m":
    model = LIGHTNING_MODEL(EFFV2M_MIXER, train_dataset, augment_dataset, test_dataset, \
                           16, 5, 16)
    check_path =  f"D:/DACON_GROWTH/EFFV2M_MIXER"
    if not os.path.exists(check_path):
        os.makedirs(check_path) 

In [46]:
checkpoint_callback = ModelCheckpoint(dirpath=check_path, save_top_k=-1,
                                      save_weights_only=True, every_n_epochs=True)
trainer = pl.Trainer(
                devices=[0],
                accelerator='gpu',
                max_epochs=70,
                callbacks=[checkpoint_callback],
                precision=16
            )

trainer.fit(model)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type               | Params
-------------------------------------------------------
0 | model           | EFFV2M_MIXER       | 51.4 M
1 | train_nmae      | NMAE               | 0     
2 | valid_nmae      | NMAE               | 0     
3 | augment_dataset | PYTORCH_AUGMENTSET | 0     
4 | train_dataset   | PYTORCH_DATASET    | 0     
5 | predict_dataset | PYTORCH_PREDICTSET | 0     
-------------------------------------------------------
51.4 M    Trainable params
0         Non-trainable params
51.4 M    Total params
102.751   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

training nmae: 0.5622, epoch: 0
training nmae: 0.3254, epoch: 1
training nmae: 0.2475, epoch: 2
training nmae: 0.2235, epoch: 3
training nmae: 0.2091, epoch: 4
training nmae: 0.1817, epoch: 5
training nmae: 0.2046, epoch: 6
training nmae: 0.2067, epoch: 7
training nmae: 0.1724, epoch: 8
training nmae: 0.1779, epoch: 9
training nmae: 0.1633, epoch: 10
training nmae: 0.1574, epoch: 11
training nmae: 0.1615, epoch: 12
training nmae: 0.1606, epoch: 13
training nmae: 0.1545, epoch: 14
training nmae: 0.1408, epoch: 15
training nmae: 0.1446, epoch: 16
training nmae: 0.1447, epoch: 17
training nmae: 0.1473, epoch: 18
training nmae: 0.1496, epoch: 19
training nmae: 0.1546, epoch: 20
training nmae: 0.1666, epoch: 21
training nmae: 0.1475, epoch: 22
training nmae: 0.1403, epoch: 23
training nmae: 0.1363, epoch: 24
training nmae: 0.1414, epoch: 25
training nmae: 0.1328, epoch: 26
training nmae: 0.1346, epoch: 27
training nmae: 0.1354, epoch: 28
training nmae: 0.1309, epoch: 29
training nmae: 0.147

efficientV2-M
epoch 55 이하 training nmae - 0.12때
->55, 54, 53, 51

efficientB0
epoch 45 이하 큰 폭 변화 비선호
->40, 41, 42


- efficient B
training nmae: 0.5622, epoch: 0
training nmae: 0.3254, epoch: 1
training nmae: 0.2475, epoch: 2
training nmae: 0.2235, epoch: 3
training nmae: 0.2091, epoch: 4
training nmae: 0.1817, epoch: 5
training nmae: 0.2046, epoch: 6
training nmae: 0.2067, epoch: 7
training nmae: 0.1724, epoch: 8
training nmae: 0.1779, epoch: 9
training nmae: 0.1633, epoch: 10
training nmae: 0.1574, epoch: 11
training nmae: 0.1615, epoch: 12
training nmae: 0.1606, epoch: 13
training nmae: 0.1545, epoch: 14
training nmae: 0.1408, epoch: 15
training nmae: 0.1446, epoch: 16
training nmae: 0.1447, epoch: 17
training nmae: 0.1473, epoch: 18
training nmae: 0.1496, epoch: 19
training nmae: 0.1546, epoch: 20
training nmae: 0.1666, epoch: 21
training nmae: 0.1475, epoch: 22
training nmae: 0.1403, epoch: 23
training nmae: 0.1363, epoch: 24
training nmae: 0.1414, epoch: 25
training nmae: 0.1328, epoch: 26
training nmae: 0.1346, epoch: 27
training nmae: 0.1354, epoch: 28
training nmae: 0.1309, epoch: 29
training nmae: 0.1473, epoch: 30
training nmae: 0.155, epoch: 31
training nmae: 0.1467, epoch: 32
training nmae: 0.1486, epoch: 33
training nmae: 0.1369, epoch: 34
training nmae: 0.1367, epoch: 35
training nmae: 0.1329, epoch: 36
training nmae: 0.1303, epoch: 37
training nmae: 0.1288, epoch: 38
training nmae: 0.1241, epoch: 39
training nmae: 0.1406, epoch: 40
training nmae: 0.1225, epoch: 41
training nmae: 0.1398, epoch: 42
training nmae: 0.1258, epoch: 43
training nmae: 0.1303, epoch: 44
training nmae: 0.1246, epoch: 45
training nmae: 0.1369, epoch: 46
training nmae: 0.1396, epoch: 47
training nmae: 0.1392, epoch: 48
training nmae: 0.1353, epoch: 49
training nmae: 0.135, epoch: 50
training nmae: 0.1275, epoch: 51
training nmae: 0.1356, epoch: 52
training nmae: 0.1235, epoch: 53
training nmae: 0.1225, epoch: 54
training nmae: 0.1239, epoch: 55
training nmae: 0.1339, epoch: 56
training nmae: 0.1178, epoch: 57
training nmae: 0.1285, epoch: 58
training nmae: 0.1337, epoch: 59
training nmae: 0.1219, epoch: 60
training nmae: 0.1255, epoch: 61
training nmae: 0.1331, epoch: 62
training nmae: 0.1238, epoch: 63
training nmae: 0.1274, epoch: 64
training nmae: 0.1225, epoch: 65
training nmae: 0.125, epoch: 66
training nmae: 0.1194, epoch: 67
training nmae: 0.1264, epoch: 68
training nmae: 0.1268, epoch: 69

-efficient b0-
training nmae: 0.6037, epoch: 0
training nmae: 0.3026, epoch: 1
training nmae: 0.2329, epoch: 2
training nmae: 0.2202, epoch: 3
training nmae: 0.2053, epoch: 4
training nmae: 0.2027, epoch: 5
training nmae: 0.1802, epoch: 6
training nmae: 0.1632, epoch: 7
training nmae: 0.1788, epoch: 8
training nmae: 0.1764, epoch: 9
training nmae: 0.1545, epoch: 10
training nmae: 0.1609, epoch: 11
training nmae: 0.1498, epoch: 12
training nmae: 0.1456, epoch: 13
training nmae: 0.1507, epoch: 14
training nmae: 0.1501, epoch: 15
training nmae: 0.1702, epoch: 16
training nmae: 0.1538, epoch: 17
training nmae: 0.1571, epoch: 18
training nmae: 0.1445, epoch: 19
training nmae: 0.1488, epoch: 20
training nmae: 0.1311, epoch: 21
training nmae: 0.1381, epoch: 22
training nmae: 0.149, epoch: 23
training nmae: 0.1401, epoch: 24
training nmae: 0.1406, epoch: 25
training nmae: 0.1329, epoch: 26
training nmae: 0.1294, epoch: 27
training nmae: 0.1384, epoch: 28
training nmae: 0.1282, epoch: 29
training nmae: 0.1491, epoch: 30
training nmae: 0.1395, epoch: 31
training nmae: 0.1264, epoch: 32
training nmae: 0.1334, epoch: 33
training nmae: 0.142, epoch: 34
training nmae: 0.1253, epoch: 35
training nmae: 0.1316, epoch: 36
training nmae: 0.1225, epoch: 37
training nmae: 0.1305, epoch: 38
training nmae: 0.1302, epoch: 39
training nmae: 0.1232, epoch: 40
training nmae: 0.125, epoch: 41
training nmae: 0.1287, epoch: 42
training nmae: 0.135, epoch: 43
training nmae: 0.1389, epoch: 44
training nmae: 0.122, epoch: 45
training nmae: 0.1235, epoch: 46
training nmae: 0.1197, epoch: 47
training nmae: 0.1215, epoch: 48
training nmae: 0.1229, epoch: 49
training nmae: 0.1324, epoch: 50
training nmae: 0.1241, epoch: 51
training nmae: 0.1292, epoch: 52
training nmae: 0.1213, epoch: 53
training nmae: 0.1251, epoch: 54
training nmae: 0.1224, epoch: 55
training nmae: 0.1147, epoch: 56
training nmae: 0.1283, epoch: 57
training nmae: 0.1217, epoch: 58
training nmae: 0.111, epoch: 59
training nmae: 0.1266, epoch: 60
training nmae: 0.1246, epoch: 61
training nmae: 0.1159, epoch: 62
training nmae: 0.1156, epoch: 63
training nmae: 0.1161, epoch: 64
training nmae: 0.1113, epoch: 65
training nmae: 0.1223, epoch: 66
training nmae: 0.1238, epoch: 67
training nmae: 0.1152, epoch: 68
training nmae: 0.1144, epoch: 69

In [61]:
ensemble1 = 0.

ensemble_path = [
f"D:/DACON_GROWTH/EFFV2M_MIXER/epoch=51-step=5460.ckpt",
f"D:/DACON_GROWTH/EFFV2M_MIXER/epoch=53-step=5670.ckpt",
f"D:/DACON_GROWTH/EFFV2M_MIXER/epoch=54-step=5775.ckpt",
f"D:/DACON_GROWTH/EFFV2M_MIXER/epoch=55-step=5880.ckpt",
]

for check in ensemble_path :
    model = LIGHTNING_MODEL.load_from_checkpoint(checkpoint_path=check, \
                model= EFFV2M_MIXER, train_datasest = None, val_dataset = None, \
                    augment_dataset= None, predict_dataset = test_dataset,\
                train_batch=16, augment_batch=5, predict_batch=16)
    prediction = trainer.predict(model)
    sub_list = []
    for p in prediction:
        p = p.view(-1)
        for sample in p:
            sample = sample.cpu().numpy()
            sub_list.append(sample)
    sub = np.array(sub_list)
    ensemble1 += sub/4

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 105it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 105it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 105it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 105it [00:00, ?it/s]

In [55]:
ensemble2 = 0.

ensemble_path = [
f"D:/DACON_GROWTH/EFFB0_MIXER/epoch=40-step=2173.ckpt",
f"D:/DACON_GROWTH/EFFB0_MIXER/epoch=41-step=2226.ckpt",
f"D:/DACON_GROWTH/EFFB0_MIXER/epoch=42-step=2279.ckpt",
]

for check in ensemble_path :
    model = LIGHTNING_MODEL.load_from_checkpoint(checkpoint_path=check, \
                model= EFFB0_MIXER, train_datasest = None, val_dataset = None, \
                    augment_dataset= None, predict_dataset = test_dataset,\
                train_batch=16, augment_batch=5, predict_batch=16)
    prediction = trainer.predict(model)
    sub_list = []
    for p in prediction:
        p = p.view(-1)
        for sample in p:
            sample = sample.cpu().numpy()
            sub_list.append(sample)
    sub = np.array(sub_list)
    ensemble2 += sub/3

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 105it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 105it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 105it [00:00, ?it/s]

In [74]:
np_nmae(ensemble1, ensemble2)

0.07446

In [82]:
b_neg = pd.read_csv(r"D:/DACON_GROWTH/prediction_collection/eff0_baseline_check_data_driven_negative2.csv")
b_= pd.read_csv(r"D:/DACON_GROWTH/prediction_collection/eff0_baseline.csv")
print(np_nmae(b_neg["leaf_weight"], ensemble2*0.97))
print(np_nmae(b_["leaf_weight"], ensemble2*0.97))

0.17646198647460054
0.1591640053475045


In [86]:
sample_submission = pd.read_csv("D:/DACON_GROWTH/sample_submission.csv")
sample_submission["leaf_weight"] = ensemble1 * 0.97
sample_submission.to_csv("D:/DACON_GROWTH/prediction_collection/effV2_mixer.csv", index=False)



In [83]:
result = ((ensemble1 + ensemble2) / 2) * 0.97

In [84]:
sample_submission = pd.read_csv("D:/DACON_GROWTH/sample_submission.csv")
sample_submission["leaf_weight"] = result
sample_submission.to_csv("D:/DACON_GROWTH/prediction_collection/ensemble_eff0_effV2_mixer.csv", index=False)


In [79]:
temp_neg = pd.read_csv(r"D:/DACON_GROWTH/prediction_collection/effV2_rw_m_negative97.csv")
temp_ori = pd.read_csv(r"D:/DACON_GROWTH/prediction_collection/effV2_rw_m_base.csv")
temp_en = pd.read_csv(r"D:/DACON_GROWTH/prediction_collection/effV2_eff0_negative_ensemble.csv")
print(np_nmae(temp_ori["leaf_weight"], ensemble1*0.96))
print(np_nmae(temp_neg["leaf_weight"], ensemble1*0.96))
print(np_nmae(temp_en["leaf_weight"], ensemble1*0.96))


0.07264489090704987
0.07377572322383238
0.08908733342840705


In [54]:
def np_nmae(y_true, y_pred):
    result = np.mean(np.abs(y_true-y_pred)) / np.mean(np.abs(y_true))
    return result

class PYTORCH_TTASET(nn.Module):
    def __init__(self, path: list,  imgs: np.array,  \
                 transform=None):
        super(PYTORCH_TTASET, self).__init__()
        self.path = path
        self.imgs = imgs

        self.transform = transform

    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        before_img = self.imgs[idx]
        img_path = self.path[idx]
        img_path = img_path.split("\\")[-1].split(".")[0]
        df_index = torch.FloatTensor(torch.zeros([49,19]))
        after_img = self.transform(image=before_img)["image"]
        return {"사진": after_img, "정보": df_index}

test_dataset2 = PYTORCH_TTASET(path=test_path, imgs=test_imgs, \
                                    transform=test_mode)

check_paths = glob(f"D:/DACON_GROWTH/EFFB0_MIXER/*")

tta_diff = {}


for check in check_paths:
    model = LIGHTNING_MODEL.load_from_checkpoint(checkpoint_path=check, \
                model= EFFB0_MIXER, train_datasest = None, val_dataset = None, \
                    augment_dataset= None, predict_dataset = test_dataset, \
                    train_batch = 16, augment_batch= 5, predict_batch= 16)
    prediction = trainer.predict(model)
    sub_list = []
    for p in prediction:
        p = p.view(-1)
        for sample in p:
            sample = sample.cpu().numpy()
            sub_list.append(sample)
    sub = np.array(sub_list)
    #TTA
    model = LIGHTNING_MODEL.load_from_checkpoint(checkpoint_path=check, \
        model = EFFB0_MIXER, train_datasest = None, val_dataset = None, \
                    augment_dataset= None, predict_dataset = test_dataset2, \
                    train_batch = 16, augment_batch= 5, predict_batch= 16)
    prediction = trainer.predict(model)
    sub_list = []
    for p in prediction:
        p = p.view(-1)
        for sample in p:
            sample = sample.cpu().numpy()
            sub_list.append(sample)
    sub2 = np.array(sub_list)
    
    result = np_nmae(sub, sub2)
    tta_diff[check] = result
    
tta_diff

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 105it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 105it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 105it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 105it [00:00, ?it/s]

{'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=0-step=53.ckpt': 0.0647,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=1-step=106.ckpt': 0.0941,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=10-step=583.ckpt': 0.2732,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=11-step=636.ckpt': 0.2478,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=12-step=689.ckpt': 0.2478,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=13-step=742.ckpt': 0.251,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=14-step=795.ckpt': 0.2869,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=15-step=848.ckpt': 0.256,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=16-step=901.ckpt': 0.2788,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=17-step=954.ckpt': 0.2766,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=18-step=1007.ckpt': 0.2218,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=19-step=1060.ckpt': 0.272,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=2-step=159.ckpt': 0.118,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=20-step=1113.ckpt': 0.2866,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=21-step=1166.ckpt': 0.2705,
 'D:/DACON_GROWTH/EFFB0_MIXER\\epoch=22-step